# **Definição do problema**
A bolsa de valores é muito dinâmica, o preço das ações muda a cada segundo e é quase impossível acompanhar. Por isso pensei em uma solução para auxiliar os investidores a monitorar sua carteira no intraday.

# **Minha proposta**
Um código que monitora os preços das ações para uma possível venda. 
O aplicativo notifica com um popup na tela do Windows, quando uma ação cumpre dois requisitos:

1.   A ação precisa ter uma intenção de venda, ou seja, o dono da carteira quer vende la;
2.   Para a venda acontecer não pode haver prejuízo, o preço atual da ação precisa ser maior do que o preço médio;

O código ainda é manual, então é necessário roda lo sempre para que funcione.

# **Arquivo excel com sua carteira de ações**
É necessário criar um arquivo excel com as ações do seu portfólio, contendo três campos:
*   ticker: código da ação na bolsa;
*   pm: preço médio de compra da ação;
*   plano: define a intenção em vender ou manter uma ação - Manter ou Vender;

# **Importando bibliotecas e subindo o arquivo**

In [25]:
#Usado para subir o arquivo
import pandas as pd

#Trabalhar com datas
from datetime import datetime

#Importar as cotações das ações com a biblioteca do Yahoo
import yfinance as yf

#Gera notificação através no windows
from winotify import Notification
import time

#Carregar o arquivo
wallet = pd.read_excel("C:/Users/Antonio/OneDrive/Área de Trabalho/python/bases/lista_acao.xlsx", sheet_name='base', header=0)
wallet.head(3)

,ticker,pm,plano
0,VALE3.SA,66.254571,Manter
1,PETR4.SA,27.940846,Manter
2,OIBR3.SA,3.250000,Vender


In [26]:
#Ticker column transformed in a list
ticker_list = wallet['ticker'].to_list()

In [27]:
#Extract from yahoo finance, the stock prices that are in your wallet
start_data = datetime.now().strftime('%Y-%m-%d')
price_quote = yf.download(ticker_list, start = start_data, rounding = True)
price_quote

C:\Users\Antonio\AppData\Local\Temp\ipykernel_10248\1195271102.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price_quote = yf.download(ticker_list, start = start_data, rounding = True)
[*********************100%***********************]  6 of 6 completed


Price         Close                                                  High  \
Ticker     COGN3.SA ELET6.SA MRVE3.SA OIBR3.SA PETR4.SA VALE3.SA COGN3.SA   
Date                                                                        
2025-10-20     3.29    58.11     6.59     0.39    29.76    61.08     3.33   

Price                                  ...     Open                    \
Ticker     ELET6.SA MRVE3.SA OIBR3.SA  ... MRVE3.SA OIBR3.SA PETR4.SA   
Date                                   ...                              
2025-10-20    58.55      6.7     0.39  ...     6.49     0.38     29.7   

Price                  Volume                                                 
Ticker     VALE3.SA  COGN3.SA ELET6.SA MRVE3.SA OIBR3.SA  PETR4.SA  VALE3.SA  
Date                                                                          
2025-10-20    60.14  28171600   709200  8555900  1258500  29175100  15517400  

[1 rows x 30 columns]

In [28]:
#Adjustment of the stock price table
price_quote = price_quote['Close']
price_quote = price_quote.reset_index()
price_quote.drop('Date', axis=1, inplace=True)
price_quote = price_quote.T
price_quote = price_quote.reset_index()
price_quote.columns = ['ticker','preco']

In [29]:
#Merging the wallet table with the stock prices table
price_quote = price_quote.merge(wallet, how = 'inner', on = 'ticker')

In [30]:
#Creating a new column with a condition
status = []
for index, row in price_quote.iterrows():
    if row['preco'] > row['pm']:
      status.append("Vender")
    else:
      status.append("Manter")

In [31]:
#Just checking the new column
price_quote['status'] = status
price_quote.head(6)

,ticker,preco,pm,plano,status
0,COGN3.SA,3.29,2.700000,Vender,Vender
1,ELET6.SA,58.11,38.692000,Manter,Vender
2,MRVE3.SA,6.59,5.130000,Vender,Vender
3,OIBR3.SA,0.39,3.250000,Vender,Manter
4,PETR4.SA,29.76,27.940846,Manter,Vender
5,VALE3.SA,61.08,66.254571,Manter,Manter


# **Notificação no Windows**
Verificação das regras e notificação no windows

In [36]:
#Price alert is sent through windows
linha = 0
while (linha < 1):
 for index, row in price_quote.iterrows():
   if((row['plano'] == 'Vender') and (row['status'] == 'Vender')):
     alerta = Notification(app_id = "Alerta de Ações",
                           title = "Oportunidade de Venda",
                           msg = "O preço atual da " + row['ticker'] +" é " + str(round(row['preco'],2)),
                           duration = "short",
                           launch = "https://br.financas.yahoo.com/portfolio/p_0/view/v2")
     alerta.show()
     time.sleep(5)
     linha = linha + 1

# **Próximos passos**
Tenho a intenção de melhorar o código continuamente:
1. Fazer com que o código rode de tempos em tempos ou com um trigger;
2. Notificação por email;

Caso tenha alguma sugestão, envie para mim!!!
Espero que tenham curtido. Abs